In [7]:
import ipywidgets as widgets
import ipyvuetify as v
import multi_component_print as mc_print
import group_component_print as gc_print
from component_filegen import gen_file
import variable_recognition as vr

def print_click(widget, event, data):
    print_but.children=['Code printed']
    print("Type:")
    print(gc.v_model)
    
    in_str = function.v_model
    
    if gc.v_model == 'om.Group':
        result = gc_print.total_parse(in_str)
        gc_print.print_all(result)
        
    elif gc.v_model == 'om.Component':
        mc_print.print_components(in_str)

def gen_click(widget, event, data):
    gen_but.children=['File generated ']
    
    inp, out = vr.get_variables(function.v_model)
    inputs = []
    outputs = []
    units_i = []
    units_o = []
    print("Inputs detected:")
    print(inp)
    print("Outputs detected:")
    print(out)
    for x in inp:
        units_i.append([1,0])
        inputs.append([x, x])
    for x in out:
        units_o.append([1,0])
        outputs.append([x, x])
    edited_function = vr.edit_function(inputs, outputs, function.v_model)
    gen_file(f_name.v_model, c_name.v_model, inputs, outputs, units_i, units_o, edited_function)
    

print_but = v.Btn(children=['Print code'])
print_but.on_event('click', print_click)
gen_but = v.Btn(children=['Generate file'], color="grey")
gen_but.on_event('click', gen_click)
gc = v.Select( v_model = 'om.Component',
    label='Type',
    items=['om.Group', 'om.Component'])
#f_name = v.TextField(v_model= 'default', label='File Name', placeholder='no extension required')
#c_name = v.TextField(v_model= 'default', label='Class Name', placeholder='starts uppercase')

function = v.Textarea(v_model= '# Component1\ny = x + 1', label='Equations' )

W = [gc, function, print_but, gen_but]

widgets.VBox(W)

In [4]:
"""
---Test Text---
## Group1

# Component1

x = 2*y + 3*z
a = b + c

# Component2

long_variable = a + b
function_test = np.log(x + np.sin(y))

## Group2

# Component3

x = 2*y + 3*z
a = b + c

# Component4

long_variable = a + b
function_test = np.log(x + np.sin(y))

"""

'\n---Test Text---\n## Group1\n\n# Component1\n\nx = 2*y + 3*z\na = b + c\n\n# Component2\n\nlong_variable = a + b\nfunction_test = np.log(x + np.sin(y))\n\n## Group2\n\n# Component3\n\nx = 2*y + 3*z\na = b + c\n\n# Component4\n\nlong_variable = a + b\nfunction_test = np.log(x + np.sin(y))\n\n'

In [ ]:
"""
---Test Text---

# Component1

x = 2*y + 3*z
a = b + c

# Component2

long_variable = a + b
function_test = np.log(x + np.sin(y))


"""